In [1]:
import pandas as pd
import os
import numpy as np
import cv2
from tkinter import Tk, Label, Button, messagebox, simpledialog
from PIL import Image, ImageTk
import xlsxwriter
from datetime import datetime

In [2]:
ROOT_DIR = os.path.abspath('')
DATA_DIR = os.path.join(ROOT_DIR, 'data')
IMAGES_DIR = os.path.join(ROOT_DIR, 'New_Billets')
input_file_path = os.path.join(DATA_DIR, 'serial_numbers_test.xlsx')
output_file_path = os.path.join(DATA_DIR, f'{datetime.now().strftime("%d-%m_%Hh_%Mm_%Ss")}_updated_serial_numbers.xlsx')

# Parameters for size of window
H = 30 # Horizontal padding in pixels
V = 40 # Vertical padding in pixels

In [3]:
df = pd.read_excel(input_file_path)
corrected_df = df.copy()
# Ensure the serial_number column is treated as strings
corrected_df['serial_number'] = corrected_df['serial_number'].astype(str)


In [4]:
# Function to rotate the image
def rotate_image(image, angle, center):
    (h, w) = image.shape[:2]
    rot_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, rot_matrix, (w, h))

# Function to calculate the window for cropping
def calculate_window(row, H, V, image_shape):
    # Ensure "C1" and "C11" are not NaN
    if pd.isna(row["C1"]) or pd.isna(row["C11"]):
        print(f"Skipping row {current_index} due to missing coordinates in 'C1' or 'C11'.")
        return None  # Signal to skip this row
    
    new_c1 = eval(row["C1"].replace(";", ","))
    new_c11 = eval(row["C11"].replace(";", ","))

    angle = np.degrees(np.arctan2(new_c11[1] - new_c1[1], new_c11[0] - new_c1[0]))
    image_center = (image_shape[1] // 2, image_shape[0] // 2)

    def rotate_point(point, angle, center):
        rot_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        return np.dot(rot_matrix, np.array([point[0], point[1], 1]))

    rotated_new_c1 = rotate_point(new_c1, angle, image_center)
    rotated_new_c11 = rotate_point(new_c11, angle, image_center)

    x_min_rotated = int(rotated_new_c1[0] - H)
    x_max_rotated = int(rotated_new_c11[0] + H)
    y_min_rotated = int(rotated_new_c1[1] - V)
    y_max_rotated = int(rotated_new_c1[1] + V)

    return rotated_new_c1, rotated_new_c11, x_min_rotated, x_max_rotated, y_min_rotated, y_max_rotated, angle

# Save the updated DataFrame
def save_data():
    corrected_df['serial_number'] = corrected_df['serial_number'].astype(str).replace('nan', '')
    corrected_df.to_excel(output_file_path, index=False, engine='xlsxwriter')
    print("Corrected data saved.")


In [5]:
# Define button actions
def update_image_display():
    global image_tk, current_index

    # Get the current row data
    row = corrected_df.iloc[current_index]
    image_path = os.path.join(IMAGES_DIR, str(row['image']))
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image at {image_path}")
        return

    # Calculate rotated and cropped image
    window_data = calculate_window(row, H, V, image.shape)
    if window_data is None:  # Skip if window calculation failed
        next_image()
        return

    rotated_new_c1, rotated_new_c11, x_min, x_max, y_min, y_max, angle = window_data
    center = (image.shape[1] // 2, image.shape[0] // 2)
    rotated_image = rotate_image(image, angle, center)

    x_min, x_max = max(0, x_min), min(rotated_image.shape[1], x_max)
    y_min, y_max = max(0, y_min), min(rotated_image.shape[0], y_max)
    cropped_image = rotated_image[y_min:y_max, x_min:x_max]

    # Convert OpenCV image to Tkinter-compatible format
    image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    image_pil = Image.fromarray(image_rgb)
    image_tk = ImageTk.PhotoImage(image_pil)  # Update global reference

    # Update the Label widget with the new image and keep the reference
    image_label.config(image=image_tk)
    image_label.image = image_tk  # Explicitly store reference
    root.title(f"Processing Image - Row {current_index}")

def on_agree():
    global current_index
    print(f"Accepted for index {current_index}")
    next_image()

def on_edit():
    global current_index
    new_serial = simpledialog.askstring("Input", "Enter the new 11-character serial number:", parent=root)
    while new_serial is not None and len(new_serial) != 11:
        messagebox.showwarning("Warning", "Invalid input. Enter exactly 11 characters.")
        new_serial = simpledialog.askstring("Input", "Enter the new 11-character serial number:", parent=root)

    if new_serial is not None:
        corrected_df.at[current_index, 'serial_number'] = new_serial
        print(f"Updated serial number for index {current_index}: {new_serial}")
    next_image()

def on_quit():
    save_data()
    root.destroy()  # Properly close Tkinter

def next_image():
    global current_index
    current_index += 1
    if current_index < len(corrected_df):
        update_image_display()  # Update display for the next image
    else:
        save_data()
        root.quit()  # Exit the Tkinter main loop
        root.destroy()  # Close when done

In [6]:
image_tk = None
current_index = 0

# Initialize the Tkinter window once
root = Tk()
root.title("Image Review")
image_label = Label(root)
image_label.grid(row=0, column=0, columnspan=3, padx=10, pady=10)

# Create buttons once and set commands
agree_button = Button(root, text="Agree", command=on_agree)
agree_button.grid(row=1, column=0, padx=10, pady=10)

edit_button = Button(root, text="Edit", command=on_edit)
edit_button.grid(row=1, column=1, padx=10, pady=10)

quit_button = Button(root, text="Quit", command=on_quit)
quit_button.grid(row=1, column=2, padx=10, pady=10)


# Start with the first image
update_image_display()
root.mainloop()

Accepted for index 0
Accepted for index 1
Accepted for index 2
Accepted for index 3
Accepted for index 4
Skipping row 5 due to missing coordinates in 'C1' or 'C11'.
Skipping row 6 due to missing coordinates in 'C1' or 'C11'.
Skipping row 7 due to missing coordinates in 'C1' or 'C11'.
Skipping row 8 due to missing coordinates in 'C1' or 'C11'.
Accepted for index 9
Corrected data saved.
